# Introduction

In today's fast-moving music industry, where streaming platforms determine much of a song's success, understanding what makes a song popular has become more important than ever. Our final project aims to take a data driven approach to analyze different factors that influence a song's popularity and aims to build a model that can predict what makes a song popular and why. 

# About the Data

We will be using a dataset from Kaggle that contains over 13,000 songs along with several parameters and measurable features. Each song entry includes both quantitative features, such as acousticness, danceability, energy, instrumentalness, loudness, speechiness, tempo, and valence, as well as metadata like genre, artist name, and release year. The target variable in this dataset is the song's popoularity score that represents how well the song performed.

We believed this dataset was a great set to use for regression modeling. Due to the mix of numerical and categorical variables, we will be using 3 different regression models (Linear, Random Forest, Gradient Boosting) to evaluate the different performances. We plan to test and compare these models to see which best captures the relationship between the different parameters and listener engagement.

# Our Goal

Our goal is to do more than build a model, we want more insight into song's popularity. We aim to answer which features matter most? Are there any unexpected patterns in what makes a song rise in popularity? Which genres tend to produce the most popular songs? 

The ultimate goal though is to not just predict song success, but to uncover any unexpected patterns and translate those into meaningful insights. This data could be valuable for several different industries and fields, increasing more hit songs. We hope to gain a deeper understanding of popularity, trends, and what shapes the music we all know and listen to today.